In [1]:
import torch, torchvision
from torch import optim, nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt

In [2]:
T=torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
mnist_data=torchvision.datasets.MNIST('mnist_data',train=True,transform=T,download=True)  
mnist_test=torchvision.datasets.MNIST('mnist_data',train=False,transform=T,download=True)  

mnist_dataloader= torch.utils.data.DataLoader(mnist_data,batch_size=64)
mnist_testloader= torch.utils.data.DataLoader(mnist_test,batch_size=64)

In [3]:
class cnn_net(nn.Module):
    def __init__(self,input_channel,hidden_channel,output_channel):
        super(cnn_net,self).__init__()
        self.conv1=nn.Conv2d(input_channel,hidden_channel,kernel_size=5)
        self.conv2=nn.Conv2d(hidden_channel,output_channel,kernel_size=5)
        self.mp=nn.MaxPool2d(2)
        self.fc=nn.Linear(320,10)
    def forward(self,x):
        in_size=x.size(0)
        x=F.relu(self.mp(self.conv1(x)))
        x=F.relu(self.mp(self.conv2(x)))
        x=x.view(in_size,-1)
        x=self.fc(x)
        x=F.softmax(x,1)
        return x

In [4]:
model=cnn_net(input_channel=1,hidden_channel=10,output_channel=20)
loss=nn.MSELoss()
params=model.parameters()
opt=optim.SGD(params=params, lr=0.01, momentum = 0.9)
print(model)

cnn_net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=320, out_features=10, bias=True)
)


In [5]:
def change(labels):
    size,length=labels.shape
    x=torch.zeros(size,10)
    for i in range (size):
        k=labels[i].cpu().numpy()
        x[i,k]=1
    return x

In [6]:
n_epochs=1
for e in range(n_epochs):
    lo=np.zeros(n_epochs)
    print('epoch=',e,end=' ')
    for i,(images,labels) in enumerate(mnist_dataloader):
        images=Variable(images)
        labels=Variable(labels.view(labels.size(0),-1))
        labels=change(labels)
        output=model(images)
        model.zero_grad()
        l=loss(output,labels)
        lo[e]=l
        l.backward()
        opt.step()
        #print(i,end=" ")
    print('loss=',lo[e])


epoch= 0 loss= 0.08075815439224243


In [7]:
labels.shape[0]

32

In [16]:
def show(x):
    x=x.detach().numpy()
    maximum=0
    for i in range (10):
        if(x[i]>maximum):
            out=i
            maximum=x[i]
    return out, maximum*100 

In [17]:
def plot(images,labels,output,i):
    images=images.view(images.size(0),-1)
    images=images.data.view(-1,28,28)
    out, maximum=show(output[i])
    print(labels[i],out,maximum)
    img2=np.array(images.detach().numpy())[i]
    plt.imshow(img2,cmap='gray')
    plt.show()
    plt.close()

In [18]:
def accuracy(output,labels,images):
    count=0
    n=labels.shape[0]
    for i in range (n):
        out, maximum=show(output[i])
        if(out==labels[i]):
            count+=1
        #else:
            #plot(images,labels,output,i)
    return count/n*100

In [19]:
a=0
x=0
for i,(images,labels) in enumerate(mnist_dataloader):
    output=model(images)
    a=accuracy(output,labels,images)
    x+=a
    #print(a,' ',x/(i+1))

In [20]:
x/(i+1)

13.69833333333334